In [1]:
def set_paths():
    '''
    Deployment should be something like "dirs/dir1/use-cases"
    This function adds to the path "dirs/dir1/use-cases" and "dirs/dir1/"
    :return:
    '''
    import imp
    from os.path import dirname
    import os
    import sys

    USE_CASES = "/var/SP/data/home/asaezco/src/devel2/use-cases"#dirname(os.path.abspath(imp.find_module('churn')[1]))
    sys.path.append("/var/SP/data/home/asaezco/src/devel2/pykhaos")
    if USE_CASES not in sys.path:
        sys.path.append(USE_CASES)
        print("Added '{}' to path".format(USE_CASES))

    # if deployment is correct, this path should be the one that contains "use-cases", "pykhaos", ...
    # FIXME another way of doing it more general?
    DEVEL_SRC = os.path.dirname(USE_CASES)  # dir before use-cases dir
    if DEVEL_SRC not in sys.path:
        sys.path.append(DEVEL_SRC)
        print("Added '{}' to path".format(DEVEL_SRC))
set_paths()

Added '/var/SP/data/home/asaezco/src/devel2/use-cases' to path
Added '/var/SP/data/home/asaezco/src/devel2' to path


In [2]:
# coding=utf-8

import sys

from common.src.main.python.utils.hdfs_generic import *
import argparse
import os
import sys
import time
from pyspark.sql.functions import (udf,
                                    col,
                                    decode,
                                    when,
                                    lit,
                                    lower,
                                    concat,
                                    translate,
                                    count,
                                    sum as sql_sum,
                                    max as sql_max,
                                    min as sql_min,
                                    avg as sql_avg,
                                    greatest,
                                    least,
                                    isnull,
                                    isnan,
                                    struct, 
                                    substring,
                                    size,
                                    length,
                                    year,
                                    month,
                                    dayofmonth,
                                    unix_timestamp,
                                    date_format,
                                    from_unixtime,
                                    datediff,
                                    to_date, 
                                    desc,
                                    asc,
                                    countDistinct,
                                    row_number,
                                    skewness,
                                    kurtosis,
                                    concat_ws)

from pyspark.sql import Row, DataFrame, Column, Window
from pyspark.sql.types import DoubleType, StringType, IntegerType, DateType, ArrayType
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer, VectorAssembler, SQLTransformer, OneHotEncoder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from datetime import datetime
from itertools import chain
import numpy as np
from functools import reduce
from utils_general import *
from pykhaos.utils.date_functions import convert_to_date
from utils_model import *
from utils_fbb_churn import *
from metadata_fbb_churn import *
from feature_selection_utils import *
import subprocess


In [3]:
# Para que no haga falta recargar el notebook si hacemos un cambio en alguna de las funciones que estamos importando
%load_ext autoreload
%autoreload 2

In [4]:
# set BDP parameters
def setting_bdp(min_n_executors = 3, max_n_executors = 15, n_cores = 8, executor_memory = "12g", driver_memory="8g",
                   app_name = "Python app", driver_overhead="1g", executor_overhead='3g'):

    MAX_N_EXECUTORS = max_n_executors
    MIN_N_EXECUTORS = min_n_executors
    N_CORES_EXECUTOR = n_cores
    EXECUTOR_IDLE_MAX_TIME = 120
    EXECUTOR_MEMORY = executor_memory
    DRIVER_MEMORY = driver_memory
    N_CORES_DRIVER = 1
    MEMORY_OVERHEAD = N_CORES_EXECUTOR * 2048
    QUEUE = "root.BDPtenants.es.medium"
    BDA_CORE_VERSION = "1.0.0"

    SPARK_COMMON_OPTS = os.environ.get('SPARK_COMMON_OPTS', '')
    SPARK_COMMON_OPTS += " --executor-memory %s --driver-memory %s" % (EXECUTOR_MEMORY, DRIVER_MEMORY)
    SPARK_COMMON_OPTS += " --conf spark.shuffle.manager=tungsten-sort"
    SPARK_COMMON_OPTS += "  --queue %s" % QUEUE

    # Dynamic allocation configuration
    SPARK_COMMON_OPTS += " --conf spark.dynamicAllocation.enabled=true"
    SPARK_COMMON_OPTS += " --conf spark.shuffle.service.enabled=true"
    SPARK_COMMON_OPTS += " --conf spark.dynamicAllocation.maxExecutors=%s" % (MAX_N_EXECUTORS)
    SPARK_COMMON_OPTS += " --conf spark.dynamicAllocation.minExecutors=%s" % (MIN_N_EXECUTORS)
    SPARK_COMMON_OPTS += " --conf spark.executor.cores=%s" % (N_CORES_EXECUTOR)
    SPARK_COMMON_OPTS += " --conf spark.dynamicAllocation.executorIdleTimeout=%s" % (EXECUTOR_IDLE_MAX_TIME)
    # SPARK_COMMON_OPTS += " --conf spark.ui.port=58235"
    SPARK_COMMON_OPTS += " --conf spark.port.maxRetries=100"
    SPARK_COMMON_OPTS += " --conf spark.app.name='%s'" % (app_name)
    SPARK_COMMON_OPTS += " --conf spark.submit.deployMode=client"
    SPARK_COMMON_OPTS += " --conf spark.ui.showConsoleProgress=true"
    SPARK_COMMON_OPTS += " --conf spark.sql.broadcastTimeout=1200"
    SPARK_COMMON_OPTS += " --conf spark.yarn.executor.memoryOverhead={}".format(executor_overhead)
    SPARK_COMMON_OPTS += " --conf spark.yarn.executor.driverOverhead={}".format(driver_overhead)

    BDA_ENV = os.environ.get('BDA_USER_HOME', '')

    # Attach bda-core-ra codebase
    SPARK_COMMON_OPTS+=" --files {}/scripts/properties/red_agent/nodes.properties,{}/scripts/properties/red_agent/nodes-de.properties,{}/scripts/properties/red_agent/nodes-es.properties,{}/scripts/properties/red_agent/nodes-ie.properties,{}/scripts/properties/red_agent/nodes-it.properties,{}/scripts/properties/red_agent/nodes-pt.properties,{}/scripts/properties/red_agent/nodes-uk.properties".format(*[BDA_ENV]*7)

    os.environ["SPARK_COMMON_OPTS"] = SPARK_COMMON_OPTS
    os.environ["PYSPARK_SUBMIT_ARGS"] = "%s pyspark-shell " % SPARK_COMMON_OPTS

In [5]:
def get_spark_session(app_name="default name", log_level='INFO', min_n_executors = 3, max_n_executors = 15, n_cores = 4, executor_memory = "12g", driver_memory="8g"):
    HOME_SRC = os.path.join(os.environ.get('BDA_USER_HOME', ''), "src")
    if HOME_SRC not in sys.path:
        sys.path.append(HOME_SRC)


    setting_bdp(app_name=app_name, min_n_executors = min_n_executors, max_n_executors = max_n_executors, n_cores = n_cores, executor_memory = executor_memory, driver_memory=driver_memory)
    from common.src.main.python.utils.hdfs_generic import run_sc
    sc, spark, sql_context = run_sc(log_level=log_level)


    return sc, spark, sql_context

In [6]:
def initialize(app_name, min_n_executors = 3, max_n_executors = 15, n_cores = 4, executor_memory = "12g", driver_memory="8g"):
    import time
    start_time = time.time()

    print("_initialize spark")
    #import pykhaos.utils.pyspark_configuration as pyspark_config
    sc, spark, sql_context = get_spark_session(app_name=app_name, log_level="OFF", min_n_executors = min_n_executors, max_n_executors = max_n_executors, n_cores = n_cores,
                             executor_memory = executor_memory, driver_memory=driver_memory)
    print("Ended spark session: {} secs | default parallelism={}".format(time.time() - start_time,
                                                                         sc.defaultParallelism))
    return spark

In [7]:
spark = initialize("Survival FBB ",executor_memory = "32g",min_n_executors = 6,max_n_executors = 15)


_initialize spark
Ended spark session: 1264.03534007 secs | default parallelism=2


In [8]:
def set_paths():
    '''
    Deployment should be something like "dirs/dir1/use-cases"
    This function adds to the path "dirs/dir1/use-cases" and "dirs/dir1/"
    :return:
    '''
    import imp
    from os.path import dirname
    import os

    USE_CASES = "/var/SP/data/home/asaezco/src/devel2/use-cases"#dirname(os.path.abspath(imp.find_module('churn')[1]))

    if USE_CASES not in sys.path:
        sys.path.append(USE_CASES)
        print("Added '{}' to path".format(USE_CASES))

    # if deployment is correct, this path should be the one that contains "use-cases", "pykhaos", ...
    # FIXME another way of doing it more general?
    DEVEL_SRC = os.path.dirname(USE_CASES)  # dir before use-cases dir
    if DEVEL_SRC not in sys.path:
        sys.path.append(DEVEL_SRC)
        print("Added '{}' to path".format(DEVEL_SRC))

In [9]:
set_paths()

In [18]:
import sys

from common.src.main.python.utils.hdfs_generic import *
import argparse
import os
import sys
import time
from pyspark.sql.functions import (udf,
                                    col,
                                    decode,
                                    when,
                                    lit,
                                    lower,
                                    concat,
                                    translate,
                                    count,
                                    sum as sql_sum,
                                    max as sql_max,
                                    min as sql_min,
                                    avg as sql_avg,
                                    greatest,
                                    least,
                                    isnull,
                                    isnan,
                                    struct, 
                                    substring,
                                    size,
                                    length,
                                    year,
                                    month,
                                    dayofmonth,
                                    unix_timestamp,
                                    date_format,
                                    from_unixtime,
                                    datediff,
                                    to_date, 
                                    desc,
                                    asc,
                                    countDistinct,
                                    row_number,
                                    skewness,
                                    kurtosis,
                                    concat_ws)

from pyspark.sql import Row, DataFrame, Column, Window
from pyspark.sql.types import DoubleType, StringType, IntegerType, DateType, ArrayType
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer, VectorAssembler, SQLTransformer, OneHotEncoder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from datetime import datetime
from itertools import chain
import numpy as np
from functools import reduce
from utils_general import *
from pykhaos.utils.date_functions import convert_to_date
from utils_model import *
from utils_fbb_churn import *
from metadata_fbb_churn import *
from feature_selection_utils import *
import subprocess

In [13]:
import imp
config_obj = Config('model_under_eval.yaml', internal_config_filename=os.path.join(imp.find_module('churn')[1], "config_manager", "config", "internal_config_churn.yaml"))
print("############ Config Object Created ############")
    
######################## TRAINING DF ########################
    
selcols = getIdFeats() + getCrmFeats() + getBillingFeats() + getMobSopoFeats() + getOrdersFeats()
now = datetime.now()
date_name = str(now.year) + str(now.month).rjust(2, '0') + str(now.day).rjust(2, '0')
origin = '/user/hive/warehouse/tests_es.db/jvmm_amdocs_ids_'

Reading config from file model_under_eval.yaml
Reading internal config from file /var/SP/data/home/asaezco/src/devel2/use-cases/churn/config_manager/config/internal_config_churn.yaml
----- CHECKING INPUT PARAMETERS ------
('check_args', 20190521, <type 'int'>)
('20190521', 1, '20190531')
----- INPUT PARAMETERS OK! ------
{'closing_day': 20190521,
 'cycles_horizon': 1,
 'discarded_cycles': 0,
 'internal_config_file': '/var/SP/data/home/asaezco/src/devel2/use-cases/churn/config_manager/config/internal_config_churn.yaml',
 'labeled': False,
 'level': 'service',
 'mode': 'test/validation/predict',
 'model_target': 'port',
 'save_car': False,
 'save_results': True,
 'segment_filter': 'mobileandfbb',
 'service_set': 'mobile',
 'services': True,
 'sources': {'ids': {'address': False,
                     'billing': True,
                     'call_centre_calls': False,
                     'campaigns': True,
                     'customer': True,
                     'customer_aggregations': 

In [23]:
selcols = getIdFeats() + getCrmFeats() + getBillingFeats() + getMobSopoFeats() + getOrdersFeats()

In [25]:
trcycle_ini = '20181130'
horizon = 4
    # Cycle used for CAR and Extra Feats in the test set
ttcycle_ini = '20181231'  # Test data
    
inittrdf_ini = getFbbChurnLabeledCarCycles(spark, origin, trcycle_ini, selcols, horizon)

dfExtraFeat = spark.read.parquet('/data/udf/vf_es/churn/extra_feats_mod/extra_feats/year={}/month={}/day={}'
                                         .format(int(trcycle_ini[0:4]), int(trcycle_ini[4:6]), int(trcycle_ini[6:8])))

        # Taking only the clients with a fbb service
dfExtraFeatfbb = dfExtraFeat.join(inittrdf_ini, ["num_cliente"], "leftsemi")

dfExtraFeatfbb = dfExtraFeatfbb.cache()
print "[Info Main FbbChurn] " + time.ctime() + " Count of the ExtraFeats: ", dfExtraFeatfbb.count()

[Info getCarNumClienteDf] Thu Jun 13 10:37:36 2019 Size of the original df for 20181130: 13248445
[Info FbbChurn] Thu Jun 13 10:50:23 2019 Saving origindf_tmp to HDFS
[Info getCarNumClienteDf] Thu Jun 13 10:50:23 2019 Size of the origindf after load from HDFS: 13248445
[Info getCarNumClienteDf] Thu Jun 13 10:51:40 2019 Size of the numclidf for 20181130 - Num rows: 1978186 - Num columns: 101
[Info FbbChurn] Thu Jun 13 10:58:10 2019 Saving numclidf_tmp to HDFS
[Info get_billing_df] Thu Jun 13 10:58:11 2019 Starting the preparation of billing feats
[Info get_billing_df] Thu Jun 13 10:58:25 2019 Size of the df with billing feats: 5714359
[Info getCarNumClienteDf] Thu Jun 13 10:58:44 2019 Size of the tmpdf for 20181130 - Num rows: 1978186 - Num columns: 134
[Info get_mobile_spinners_df] Thu Jun 13 10:59:10 2019 Port-out data loaded for 20181130 with a total of 27468495 rows and 11448545 distinct NIFs
[Info get_mobile_spinners_df] Thu Jun 13 10:59:25 2019 Timing feats computed for a total of

In [26]:
dfExtraFeatSel, selColumnas = addExtraFeatsEvol(dfExtraFeatfbb)

print "[Info Main FbbChurn] " + time.ctime() + " Calculating the total value of the extra feats for each number client"

dfillNa = fillNa(spark)
for kkey in dfillNa.keys():
    if kkey not in dfExtraFeatSel.columns:
        dfillNa.pop(kkey, None)

inittrdf = inittrdf_ini.join(dfExtraFeatSel, ["msisdn", "num_cliente", 'rgu'], how="left").na.fill(dfillNa)
print("############ Finished loading Training Dataset ############")

[Info FbbChurn] Thu Jun 13 11:18:14 2019 Inside CCC module of the Extra Feats 
[Info FbbChurn] Thu Jun 13 11:18:15 2019 Number of columns processed: 205
[Info FbbChurn] Number of rows: 8335417
[Info FbbChurn] Thu Jun 13 11:19:19 2019 The module for CCC has been run. Number of clients  1978186
[Info FbbChurn] Thu Jun 13 11:19:49 2019 Inside NS module of the Extra Feats 
[Info FbbChurn] Thu Jun 13 11:19:51 2019 Number of columns processed: 608
[Info FbbChurn] Thu Jun 13 11:19:58 2019 The module for NS has been run. Number of clients  1978186
[Info FbbChurn] Thu Jun 13 11:27:01 2019 Inside TGS module of the Extra Feats 
[Info FbbChurn] Thu Jun 13 11:27:02 2019 Number of columns processed: 21
[Info FbbChurn] Thu Jun 13 11:27:02 2019 The module for TGS has been run. Number of clientes  1978186
[Info FbbChurn] Thu Jun 13 11:27:25 2019 Inside Orders module of the Extra Feats 
[Info FbbChurn] Thu Jun 13 11:27:25 2019 Number of columns processed: 8
[Info FbbChurn] Thu Jun 13 11:27:26 2019 The m

In [27]:
ttdf_ini = getFbbChurnLabeledCarCycles(spark, origin, ttcycle_ini, selcols,horizon)

print "[Info Main FbbChurn] " + time.ctime() + " Saving ttdf_ini to HDFS "

dfExtraFeat_tt = spark.read.parquet('/data/udf/vf_es/churn/extra_feats_mod/extra_feats/year={}/month={}/day={}'
                                            .format(int(ttcycle_ini[0:4]), int(ttcycle_ini[4:6]), int(ttcycle_ini[6:8])))

dfExtraFeatfbb_tt = dfExtraFeat_tt.join(ttdf_ini.select('num_cliente'), on='num_cliente', how='leftsemi')
print(dfExtraFeatfbb_tt.select('num_cliente').distinct().count(), ttdf_ini.select('num_cliente').distinct().count())

dfExtraFeatfbb_tt = dfExtraFeatfbb_tt.cache()
print("[Info Main FbbChurn] " + time.ctime() + " Count of the ExtraFeats ", dfExtraFeatfbb_tt.count())

dfExtraFeat_ttSel, selColumnas = addExtraFeatsEvol(dfExtraFeatfbb_tt)

print "[Info Main FbbChurn] " + time.ctime() + " Calculating the total value of the extra feats for each number client in tt"

dfillNa = fillNa(spark)
for kkey in dfillNa.keys():
    if kkey not in dfExtraFeat_ttSel.columns:
        dfillNa.pop(kkey, None)

ttdf = ttdf_ini.join(dfExtraFeat_ttSel, ["msisdn", "num_cliente", 'rgu'], how="left").na.fill(dfillNa)
print "[Info Main FbbChurn] " + time.ctime() + " Number of clients after joining the Extra Feats to the test set " + str(ttdf.count())
print("############ Finished loading Test Dataset ############")

[Info getCarNumClienteDf] Thu Jun 13 11:53:22 2019 Size of the original df for 20181231: 13683905
[Info FbbChurn] Thu Jun 13 12:07:42 2019 Saving origindf_tmp to HDFS
[Info getCarNumClienteDf] Thu Jun 13 12:07:42 2019 Size of the origindf after load from HDFS: 13683905
[Info getCarNumClienteDf] Thu Jun 13 12:09:00 2019 Size of the numclidf for 20181231 - Num rows: 2050838 - Num columns: 101
[Info FbbChurn] Thu Jun 13 12:15:57 2019 Saving numclidf_tmp to HDFS
[Info get_billing_df] Thu Jun 13 12:15:58 2019 Starting the preparation of billing feats
[Info get_billing_df] Thu Jun 13 12:16:51 2019 Size of the df with billing feats: 5825804
[Info getCarNumClienteDf] Thu Jun 13 12:17:04 2019 Size of the tmpdf for 20181231 - Num rows: 2050838 - Num columns: 134
[Info get_mobile_spinners_df] Thu Jun 13 12:17:19 2019 Port-out data loaded for 20181231 with a total of 27844643 rows and 11499038 distinct NIFs
[Info get_mobile_spinners_df] Thu Jun 13 12:18:10 2019 Timing feats computed for a total of

Py4JJavaError: An error occurred while calling o26089.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 1611.0 failed 4 times, most recent failure: Lost task 1.3 in stage 1611.0 (TID 153312, vgddp367hr.dc.sedc.internal.vodafone.com, executor 29): java.io.FileNotFoundException: File does not exist: hdfs://nameservice1/data/udf/vf_es/churn/fbb_tmp/numclidf_tmp_20181231/part-00100-752d3347-024b-4510-b91d-6a15c1650d4b-c000.snappy.parquet
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:131)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:182)
	at org.apache.spark.sql.execution.datasourc

In [ ]:
closing_day_f = '20181231'
key = "censor"
label = 'label'
now = dt.now().strftime("%Y%m%d")

quantileProbabilities = [0.5, 0.75, 0.8, 0.9, 0.95, 0.99]
    
cycles_horizon = 4
closing_day = '20181231'
discarded_cycles = 0
start_date = '20181130'
start_date = move_date_n_cycles(start_date, n=discarded_cycles) if discarded_cycles > 0 else move_date_n_days(start_date, n=1)
end_date = move_date_n_cycles(start_date, n=cycles_horizon)
print('End Date')
print(end_date)
df_sol_por_discard = get_port_requests_table(spark, config_obj, start_date, end_date, closing_day, select_cols=['msisdn_a','portout_date', 'label'])
    
df_ids = getIds(spark, closing_day_f)
    
portas = df_sol_por_discard.select('msisdn_a','portout_date', 'label').withColumnRenamed('msisdn_a','msisdn')
    
labeled = inittrdf.join(portas, ['msisdn'], how="left").na.fill({'label': 0.0})
    
labeled2 = labeled.withColumn('endDate', labeled['portout_date'].cast(DateType()))
    
start_ = 'fx_fbb_fx_first'
    
labeled3 = labeled2.join( df_ids.select('msisdn', start_ +'_nif', start_ +'_nc'), ['msisdn'], 'inner')

In [ ]:
from pyspark.sql.functions import least, col
labeled4 = labeled3.withColumn('startTime', least(labeled3[start_ +'_nif'],labeled3[start_ +'_nc']))
prepared = labeled4.withColumn('startDate', labeled4['startTime'].cast(DateType()))
    
from pyspark.sql import functions as F
timeFmt = "yyyy-MM-dd"
timeDiff = (F.unix_timestamp('endDate', format=timeFmt) - F.unix_timestamp('startDate', format=timeFmt))
final_df = prepared.withColumn("Duration", timeDiff)

final_censor = final_df.withColumnRenamed('label', 'censor')
timeDiff2 = (F.unix_timestamp('closing_day', format=timeFmt) - F.unix_timestamp('startTime', format=timeFmt))
import datetime as datetime
closing_date=datetime.date(int(closing_day_f[:4]),int(closing_day_f[4:6]),int(closing_day_f[6:8]))
    
final = final_censor.withColumn('closing_day', lit(closing_date))
    
sec_2_day = 3600*24
    
final_2 = final.withColumn('label', when(final[key] == 1, final_censor['Duration']/(sec_2_day)).otherwise(timeDiff2/(sec_2_day)))
    
final_d = final_2
(Train_, Test_) = final_d.randomSplit([0.8, 0.2])

# Getting only the numeric variables
allFeats = inittrdf.columns
catCols = [item[0] for item in inittrdf.dtypes if item[1].startswith('string')]
numerical_feats = list(set(allFeats) - set(list(set().union(getIdFeats(), getIdFeats_tr(), getNoInputFeats(), catCols, [c + "_enc" for c in getCatFeatsCrm()],
                    ["label"]))))
    
trdf = balance_df2(Train_.where(col(label).isNotNull()).filter(Train_[label] > 0), 'censor')
print("############ Balanced Dataset ############")

In [29]:
spark.stop()